# Prioritized Experience Replay

> Fill in a module description here

In [ ]:
#| default_exp replay

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
from typing import List, Dict, Union

import torch
from torch import nn
import torch.nn.functional as F
from torch.distributions import Bernoulli, Uniform
from torchtyping import TensorType

from repaired.level import Level

### Prioritized Level Replay

In [ ]:
#| export
class PrioritizedReplayDistribution:
    def __init__(
        self,
        staleness_coeff: float = 0.1,
        temperature: float = 0.1, # the beta coefficient for the P_S distribution
    ) -> None:
        self.staleness_coeff = staleness_coeff
        self.temperature = temperature
    
    def create(
        self,
        score_levels: Dict[Level, Union[int, float]],
        last_episode_levels: Dict[Level, int], # the last episode that each level was played,
        last_episode: int # the last episode 
    ) -> TensorType["n_visited_levels"]:
        """Create a prioritized level distribution."""
        
        score_levels = torch.tensor([v for v in score_levels.values()])
        last_episode_levels = torch.tensor([v for v in last_episode_levels.values()])
        
        level_scores = torch.pow(
            input=F.normalize(score_levels, dim=-1),
            exponent=1/self.temperature
        )
        score_dist = level_scores / level_scores.sum(dim=-1)
        
        stale_scores = last_episode - last_episode_levels
        stale_dist = stale_scores / stale_scores.sum(dim=-1)
        
        prioritized_dist = (1 - self.staleness_coeff) * score_dist + self.staleness_coeff * stale_dist
        
        return prioritized_dist

In [ ]:
#| export
class PrioritizedReplay:
    def __init__(
        self,
        levels: List[Level],
    ) -> None:
        self.levels = levels
        self.visited_levels: List[Level] = []
        self.visited_count_levels: Dict[Level, int] = {}
        self.last_count_levels: Dict[Level, int] = {}
        self.score_levels: Dict[Level, Union[int, float]] = {}
        self.last_episode = 0
        
        self.prioritized_dist = PrioritizedReplayDistribution()
    
    def sample_next_level(
        self,
        visited_levels: List[Level],
        score_levels: Dict[str, Union[int, float]],
        last_episode_levels: Dict[str, int],
        last_episode: int
    ) -> Level:
        """Sampling a level from the replay distribution."""
        # sample replay decision
        decision_dist = Bernoulli(probs=0.5)
        
        # write unseen level by filter level_id
        unseen_levels = [level for level in self.levels if level not in visited_levels]

        if decision_dist.sample() == 0 and  len(unseen_levels) > 0:            
            # sample an unseen level
            uniform_dist = torch.rand(len(unseen_levels))
            selected_index = torch.argmax(uniform_dist)
            next_level = unseen_levels[selected_index]
            
            self.visited_count_levels[next_level] = 1
        else:
            # sample a level for replay
            prioritized_dist = self.prioritized_dist.create(
                score_levels,
                last_episode_levels,
                last_episode
            )
            
            visited_idx = torch.multinomial(prioritized_dist, num_samples=1)
            next_level = visited_levels[visited_idx]
            
        return next_level